### Visualize Dapi Mosaic

In [6]:
if False:
    import numpy as np
    import tifffile
    mosaic = tifffile.imread(r'Z:\MERFISHp\12_16_2025_BigSlideTest\final_spots\dapi_set9.tif')
    import napari
    V = napari.Viewer()
    #V.add_image(np.max(mosaic,axis=0))
    V.add_image(mosaic[5])

C:\Users\miket\.conda\envs\merfishplus\lib\site-packages\napari\_vispy\layers\scalar_field.py:197: UserWarning: data shape (20493, 35612) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Image layer 'Image' at 0x11aa1024910>

### Save matrices

In [ ]:
import numpy as np
iset = 2
spot_fl = rf'Z:\MERFISHp\12_16_2025_BigSlideTest\final_spots\spots_set{iset}_final.npz'
XF = np.load(spot_fl)['XF']
genes = np.load(spot_fl)['genes']
#mosaic = tifffile.imread(r'Z:\MERFISHp\12_16_2025_BigSlideTest\final_spots\dapi_set1.tif')
pix_size = [5,3,3]
Xmos = XF[:,:3]/pix_size
Xpix = XF[:,:3]
pix_sizeF = np.load(spot_fl)['pix_size']
Xum = Xpix*pix_sizeF

import pickle,os
dic_pos_fl = r'Z:/MERFISHp/12_16_2025_BigSlideTest/dic_pos_pixf.pkl'
dic_pos = pickle.load(open(dic_pos_fl,'rb'))
fovs = np.sort([fov for fov in dic_pos if f'zscan{iset}_' in fov])
all_genes = [ln.split(',')[0]for ln in open(r'Z:\MERFISHp\codebook_BRBB_500Markergn_NewAdaptors.csv') if ',' in ln][1:]


from tqdm import tqdm
for fov in tqdm(fovs):
    fl_save_mat = rf'Z:\MERFISHp\12_16_2025_BigSlideTest\final_mats\{fov}--mat.npz'
    fl_segm = rf'Z:\MERFISHp\12_16_2025_BigSlideTest\Segmentation2d\{fov}--H1_MER_set{iset}--segm.npz'
    if not os.path.exists(fl_save_mat):
        segm = np.load(fl_segm)['segm']
        shape = np.load(fl_segm)['shape']
        resc = shape/segm.shape
        Xcells = np.array(np.where(segm>0)).T
        if len(Xcells)>0:
            cell_ids = segm[tuple(Xcells.T)]
            Xcells = Xcells*resc
            Xm = np.min([dic_pos[fov] for fov in fovs],axis=0)
            Xcells[:,1:] = Xcells[:,1:]+dic_pos[fov]-Xm
            Xcells = Xcells*pix_sizeF
            Xcm = np.min(Xcells,axis=0)
            XcM = np.max(Xcells,axis=0)
            dth=2.5
        
            keep = np.all((Xum>=(Xcm-dth))&(Xum<=(XcM+dth)),axis=-1)
            
            Xum_ = Xum[keep]
            genes_ = genes[keep]
            if len(Xum_)>0:
                from scipy.spatial import KDTree
                dd,ii = KDTree(Xcells).query(Xum_,workers=5)
                keep_in_cell = dd<=dth
                cell_idsF = cell_ids[ii[keep_in_cell]]
                genesF = genes_[keep_in_cell]
                ucells,inv_cells = np.unique(cell_idsF,return_inverse=True)
            
            
                from scipy import ndimage
                XCMS_cells = np.array(ndimage.center_of_mass(segm>0,segm,ucells))
                if len(XCMS_cells)>0:
                    XCMS_cells=XCMS_cells*resc
                    XCMS_cells[:,1:]+=dic_pos[fov]-Xm
                    XCMS_cells = XCMS_cells*pix_sizeF
                
                
                    genesU,inv_genes = np.unique(np.concatenate([genesF,all_genes]),return_inverse=True)
                    inv_genes=inv_genes[:-len(all_genes)]
                    cellsU = [fov+'--'+str(cell)for cell in ucells]
                    Mcounts = np.zeros([len(cellsU),len(genesU)],dtype=np.uint32)
                    np.add.at(Mcounts,(inv_cells,inv_genes),1)
                    #np.sum(Mcounts,axis=-1)
                    np.savez_compressed(fl_save_mat,Mgenes = genesU,Mcounts=Mcounts,Mcells=cellsU,Xum=XCMS_cells)

  1%|▊                                                                               | 3/300 [00:49<1:19:50, 16.13s/it]

In [88]:
# import napari
# V = napari.view_points(Xcells[:,1:],face_color='r',size=0.5)
# V.add_points(Xum_[genes_=='Slc17a7'][:,1:],size=0.5)

In [77]:
len(inv_genes)

1274149

In [24]:
X2d = Xpix[genes=='Reln'][:,1:]

In [28]:
#V = napari.view_points(X2d,border_width=0,size=5)
V.add_points(Xcells[:,1:],border_width=0,size=5,face_color='r')

<Points layer 'Points' at 0x11ace2283a0>

In [20]:
gns,cts = np.unique(genes[::1000],return_counts=True)
gns[np.argsort(cts)][-150:]

array(['Ntn1', 'Opalin', 'Gfra1', 'Ighm', 'Gli3', 'Lmo3', 'St3gal1',
       'Hspg2', 'Col5a1', 'Man1a', 'Bmpr1b', 'Synpr', 'Slit2', 'Rprm',
       'Cd34', 'Penk', 'Fras1', 'Kctd8', 'Gda', 'Nrp1', 'Galnt14',
       'Adamts2', 'Car4', 'Cdh20', 'Foxo1', 'Oprd1', 'Tgfbr2', 'Ebf1',
       'St18', 'Cgnl1', 'Drd2', 'Agt', 'Tbx3', 'Igfbp6', 'Otof',
       'Tmem132c', 'Rab3b', 'Lpar1', 'Hs3st2', 'Ramp1', 'Ldb2', 'C1ql3',
       'Rorb', 'Wnt7b', 'Rgs6', 'Elfn1', 'Kcng1', 'Nr2f2', 'Sv2c', 'Kit',
       'Maf', 'Cnih3', 'Mdga1', 'Chrm3', 'Tafa2', 'Osbpl3', 'Sema5a',
       'Ust', 'Fibcd1', 'Mog', 'Lhfp', 'Igfbp2', 'Chn2', 'Syndig1',
       'Slc7a10', 'Nr4a3', 'Calb2', 'Syt17', 'Pvalb', 'Fxyd6', 'Trhde',
       'Lypd1', 'Crym', 'Adcy2', 'Ptprm', 'Ccn3', 'Nrn1', 'Sox2', 'Gad2',
       'Sox5', 'Shisa6', 'Nr4a2', 'Nptx2', 'Cacng3', 'St6galnac5',
       'Lamp5', 'Aqp4', 'Parm1', 'Ccnd2', 'A830036E02Rik', 'Zfp521',
       'Igfbp4', 'Zfp536', 'Hs3st4', 'Pdzrn3', 'Kcnab3', 'Megf11',
       'Serpine2', 'Bcl

In [21]:
import napari
V = napari.Viewer()
#V.add_image(np.max(mosaic,axis=0))
gene ='Opalin' 
V.add_points(Xmos[genes==gene][:,1:],border_width=0,face_color=[1,1,1,0.1],size=0.25,name=gene)

<Points layer 'Opalin' at 0x179b94aca30>